# **Bước 0: Import thư viện**

In [1]:
import keras
import os
import glob
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import shutil
from tensorflow.keras import models, layers

# **Bước 1: Đọc ảnh và file JSON**

In [2]:
# Đường dẫn thư mục dữ liệu gốc
IMAGE_DIR = "/kaggle/input/btxrd-data/BTXRD/BTXRD/images"
JSON_DIR = "/kaggle/input/btxrd-data/BTXRD/BTXRD/Annotations"

# Thư mục lưu kết quả sau khi xử lý
OUTPUT_IMAGE_DIR = "/kaggle/working/processed_images"
OUTPUT_JSON_DIR = "/kaggle/working/processed_annotations"

if not os.path.exists(OUTPUT_IMAGE_DIR):
    os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)  # Tạo thư mục nếu chưa có
    print(f"✅ Đã tạo thư mục: {OUTPUT_IMAGE_DIR}")
else:
    print(f"✅ Thư mục đã tồn tại: {OUTPUT_IMAGE_DIR}")

if not os.path.exists(OUTPUT_JSON_DIR):
    os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)  # Tạo thư mục nếu chưa có
    print(f"✅ Đã tạo thư mục: {OUTPUT_JSON_DIR}")
else:
    print(f"✅ Thư mục đã tồn tại: {OUTPUT_JSON_DIR}")

✅ Thư mục đã tồn tại: /kaggle/working/processed_images
✅ Thư mục đã tồn tại: /kaggle/working/processed_annotations


# ** Bước 2: Tiền xử lý dữ liệu**

Resize ảnh về kích thước cố định nhưng không méo hình.  
Padding ảnh để giữ nguyên tỷ lệ.  
Cập nhật tọa độ vùng ung thư trong JSON để không bị sai lệch.  

In [3]:
def resize_and_pad_image(img_path, json_path):
    """ Resize ảnh về kích thước chuẩn và cập nhật tọa độ vùng ung thư trong JSON. """
    # Đọc ảnh gốc
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape

    # Tính tỷ lệ resize để không làm méo ảnh
    scale = min(TARGET_SIZE[0] / w, TARGET_SIZE[1] / h)
    new_w, new_h = int(w * scale), int(h * scale)
    
    # Resize ảnh
    resized_img = cv2.resize(img, (new_w, new_h))

    # Tạo ảnh nền trắng để pad
    final_img = np.ones((TARGET_SIZE[1], TARGET_SIZE[0]), dtype=np.float32) * 255
    x_offset = (TARGET_SIZE[0] - new_w) // 2
    y_offset = (TARGET_SIZE[1] - new_h) // 2

    # Đưa ảnh vào trung tâm
    final_img[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_img

    # Cập nhật JSON
    with open(json_path, "r") as f:
        data = json.load(f)

    # Cập nhật tọa độ vùng tổn thương
    for shape in data["shapes"]:
        updated_points = []
        for point in shape["points"]:
            x_new = point[0] * scale + x_offset
            y_new = point[1] * scale + y_offset
            updated_points.append([x_new, y_new])

        shape["points"] = updated_points

    return final_img / 255.0, data

In [4]:
TARGET_SIZE = (1600, 2100)  # (width, height)

# Duyệt qua tất cả ảnh trong thư mục
image_paths = glob.glob(IMAGE_DIR + "/*.png") + glob.glob(IMAGE_DIR + "/*.jpg") + glob.glob(IMAGE_DIR + "/*.jpeg")

for img_path in image_paths:
    filename = os.path.basename(img_path).split('.')[0]  # Lấy tên file không có đuôi
    json_path = os.path.join(JSON_DIR, filename + ".json")  # Tìm file JSON tương ứng
    
    if os.path.exists(json_path):
        # Xử lý ảnh + tọa độ ung thư
        processed_img, updated_annotations = resize_and_pad_image(img_path, json_path)

        if processed_img is not None:
            # Lưu ảnh mới
            output_img_path = os.path.join(OUTPUT_IMAGE_DIR, filename + ".jpg")
            cv2.imwrite(output_img_path, (processed_img * 255).astype(np.uint8))  # Chuyển về 0-255

            # Lưu JSON mới
            output_json_path = os.path.join(OUTPUT_JSON_DIR, filename + ".json")
            with open(output_json_path, "w") as f:
                json.dump(updated_annotations, f, indent=4)

print("✅ Xử lý hoàn tất. Ảnh và JSON đã được lưu tại", OUTPUT_IMAGE_DIR, "và", OUTPUT_JSON_DIR)


✅ Xử lý hoàn tất. Ảnh và JSON đã được lưu tại /kaggle/working/processed_images và /kaggle/working/processed_annotations


# **Bước 3: Phân chia bộ test và train**

Phân chia ảnh thành 80% train và 20% test

In [5]:
#Thư mục lưu kết quả train và test
TRAIN_IMAGE_DIR = "/kaggle/working/train/images"
TRAIN_JSON_DIR = "/kaggle/working/train/annotations"
TEST_IMAGE_DIR = "/kaggle/working/test/images"
TEST_JSON_DIR = "/kaggle/working/test/annotations"

# Tạo thư mục nếu chưa có
for folder in [TRAIN_IMAGE_DIR, TRAIN_JSON_DIR, TEST_IMAGE_DIR, TEST_JSON_DIR]:
    os.makedirs(folder, exist_ok=True)

# Lấy danh sách tất cả ảnh
image_paths = glob.glob(OUTPUT_IMAGE_DIR + "/*.jpg") + \
              glob.glob(OUTPUT_IMAGE_DIR + "/*.png") + \
              glob.glob(OUTPUT_IMAGE_DIR + "/*.jpeg")

# Shuffle dữ liệu
random.seed(42)  # Để đảm bảo chia dữ liệu giống nhau mỗi lần chạy
random.shuffle(image_paths)

# Chia theo tỷ lệ 80% train - 20% test
split_idx = int(0.8 * len(image_paths))
train_images = image_paths[:split_idx]
test_images = image_paths[split_idx:]

# Chia dataset
for img_path in train_images:
    filename = os.path.basename(img_path)
    json_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".jpg", ".json"))

    # Di chuyển ảnh và JSON vào thư mục train
    shutil.move(img_path, os.path.join(TRAIN_IMAGE_DIR, filename))
    if os.path.exists(json_path):
        shutil.move(json_path, os.path.join(TRAIN_JSON_DIR, os.path.basename(json_path)))

for img_path in test_images:
    filename = os.path.basename(img_path)
    json_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".jpg", ".json"))

    # Di chuyển ảnh và JSON vào thư mục test
    shutil.move(img_path, os.path.join(TEST_IMAGE_DIR, filename))
    if os.path.exists(json_path):
        shutil.move(json_path, os.path.join(TEST_JSON_DIR, os.path.basename(json_path)))

print("✅ Đã chia dataset thành train/test và lưu vào thư mục.")

✅ Đã chia dataset thành train/test và lưu vào thư mục.


# **Bước 4: Xây dựng hệ thống CNN**

In [ ]:
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(2100,1600,3)))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

# Block 2
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

# Block 3
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

# Block 4
model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

# Flatten và Fully Connected Layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))  # 10 lớp phân loại

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# **Huấn luyện mô hình**